# Investigation into the results

Let's see where exactly our models succeeded and failed.

In [1]:
import pandas as pd

In [2]:
result_1 = pd.read_csv('/home/snow/NEU/CS6120/Assignments/A4/data/model_1.csv')
result_2 = pd.read_csv('/home/snow/NEU/CS6120/Assignments/A4/data/model_2.csv')
result_3 = pd.read_csv('/home/snow/NEU/CS6120/Assignments/A4/data/model_3.csv')
review_set = pd.read_csv('/home/snow/NEU/CS6120/Assignments/A4/data/review_set_latest.csv')

In [3]:
result_1.rename(columns={'sentiment_analysis': 'sentiment_analysis_1'}, inplace=True)
result_2.rename(columns={'sentiment_analysis': 'sentiment_analysis_2'}, inplace=True)
result_3.rename(columns={'sentiment_analysis': 'sentiment_analysis_3'}, inplace=True)
review_set.rename(columns={'sentiment': 'ground_truth_sentiment'}, inplace=True)

In [4]:
combined = pd.concat([result_1, result_2, result_3, review_set], axis=1)

# Models
We used the following models (fetched from huggingface):
<br><br>
model_1 = 'distilbert-base-uncased-finetuned-sst-2-english' (Result_1)<br>
model_2 = 'cardiffnlp/twitter-roberta-base-sentiment-latest' (Result_2)<br>
model_3 = 'siebert/sentiment-roberta-large-english' (Result_3)<br>

In [60]:
model_1 = 'distilbert-base-uncased-finetuned-sst-2-english'
model_2 = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
model_3 = 'siebert/sentiment-roberta-large-english'

In [5]:
combined

,sentiment_analysis_1,sentiment_analysis_2,sentiment_analysis_3,label,text,ground_truth_sentiment
0,NEGATIVE,neutral,NEGATIVE,1,possible cigar shop even less friendly fair oa...,negative
1,POSITIVE,positive,POSITIVE,3,one favourite spots tempe offers eclectic iris...,positive
2,NEGATIVE,positive,NEGATIVE,0,new mexico thought id go amazing green red chi...,negative
3,POSITIVE,positive,POSITIVE,3,food awesome drinks kind watered sitting outsi...,positive
4,POSITIVE,positive,POSITIVE,4,bourbon great place enjoy brunch crazy nights ...,positive
...,...,...,...,...,...,...
199995,NaN,NaN,NaN,1,let sum sweet cakes two words rated my wife lo...,negative
199996,NaN,NaN,NaN,1,of as exist less a blocks apartment would neve...,negative
199997,NaN,NaN,NaN,1,know seen place showcased food network a times...,negative
199998,NaN,NaN,NaN,1,understand hype lamb linda loo meh meat potato...,negative


Trimming to make sure only the columns we experimented on are retained

In [6]:
combined.dropna(inplace=True)

In [7]:
combined['sentiment_analysis_1'] = combined['sentiment_analysis_1'].str.lower()
combined['sentiment_analysis_2'] = combined['sentiment_analysis_2'].str.lower()
combined['sentiment_analysis_3'] = combined['sentiment_analysis_3'].str.lower()

Also, let's assume that neutral corresponds to negative in the second model.

In [8]:
combined['sentiment_analysis_2'] = combined['sentiment_analysis_2'].replace('neutral', 'negative')

Let's first calculate the accuracy of each model to get an initial sembalance of how "accurate" they might be for this use case

<br>

First, Model 1

In [9]:
accuracy_model_1 = (combined['sentiment_analysis_1'] == combined['ground_truth_sentiment']).mean() * 100
accuracy_model_1


80.99047923833906

It still leaves 20% of the cases, interesting

In [32]:
incorrect_1 = combined[combined['sentiment_analysis_1'] != combined['ground_truth_sentiment']]
incorrect_1.drop(columns=['sentiment_analysis_2', 'sentiment_analysis_3'], axis=1, inplace=True)
incorrect_1 = incorrect_1.reindex(columns=['label','text', 'ground_truth_sentiment', 'sentiment_analysis_1'])


/tmp/ipykernel_50355/4026146000.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incorrect_1.drop(columns=['sentiment_analysis_2', 'sentiment_analysis_3'], axis=1, inplace=True)


In [33]:
incorrect_1

,label,text,ground_truth_sentiment,sentiment_analysis_1
12,0,first good where began cannot begin known know...,negative,positive
26,4,first trip wildlife world zoo great like dirt ...,positive,negative
31,4,place truly gem east valley first cheaper chea...,positive,negative
42,3,writing first review customer service manageme...,positive,negative
45,1,decided pay homage faux jewish tradition chine...,negative,positive
...,...,...,...,...
49987,4,want live restaurants i posted go robot headin...,positive,negative
49988,1,something old spaghetti factory love really li...,negative,positive
49991,3,wife twice last two weeks four times together ...,positive,negative
49994,3,hearing place endorsed oprah friend gale best ...,positive,negative


Let's look at various outputs where the model went wrong here..

In [34]:
incorrect_1.sample(5)

,label,text,ground_truth_sentiment,sentiment_analysis_1
35533,4,love place never bad experience times one gree...,positive,negative
22539,3,wait reviewed crepes parisian penni first fell...,positive,negative
7328,1,international hub airport regional airport fee...,negative,positive
44001,1,atmosphere really nice server took forever kno...,negative,positive
22489,3,landed vegas picked friend taken dinner eat su...,positive,negative


Let's take a closer look

In [36]:
incorrect_1.text[35533]

'love place never bad experience times one green valley freshly squeezed juices amazing especially watermelon favourite one funny thing place apparently difference breakfast burrito breakfast wrap bacon lovers get wrap burrito kind slaughters animal'

Here we can see although this is describing something in irony, the model only looks at words like "bad experience".


Let's try looking at the attention scores

In [37]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

def get_top_attended_words(sentence, model_name, top_n=3):
    """
    Returns the top `top_n` words with the highest attention scores for a given sentence
    using a pre-trained transformer model from Hugging Face.

    Args:
        sentence (str): The input sentence.
        model_name (str): The name of the pre-trained Hugging Face model to use.
        top_n (int, optional): The number of top attended words to return. Defaults to 3.

    Returns:
        list: A list of tuples containing the top attended words and their attention scores.
    """
    # Load the pre-trained model and tokenizer
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt")

    # Get the attention scores from the model
    outputs = model(**inputs, output_attentions=True)
    attention_scores = outputs.attentions

    # Decode the input token IDs
    input_ids = inputs.input_ids.squeeze().tolist()
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Compute the average attention score for each word across all heads and layers
    word_scores = []
    for token_idx, token in enumerate(tokens):
        if token not in tokenizer.all_special_tokens:
            avg_score = 0
            for layer in attention_scores:
                attention_matrix = layer.squeeze(0)
                for head in range(attention_matrix.size(0)):
                    avg_score += attention_matrix[head, token_idx, token_idx].item()
            avg_score /= (len(attention_scores) * attention_matrix.size(0))
            word_scores.append((token, avg_score))

    # Sort the words by their attention scores in descending order
    word_scores = sorted(word_scores, key=lambda x: x[1], reverse=True)

    # Return the top `top_n` attended words and their scores
    return word_scores[:top_n]

/home/snow/miniconda3/envs/CS6120/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [39]:
get_top_attended_words(incorrect_1.text[35533], 'nlptown/bert-base-multilingual-uncased-sentiment', top_n=10)

[('never', 0.12569126340819123),
 ('lovers', 0.11829061958990057),
 ('difference', 0.1083284916570043),
 ('green', 0.1073714267761151),
 ('times', 0.10689252069211802),
 ('bad', 0.1054180827335735),
 ('favourite', 0.10424478334484395),
 ('apparently', 0.09845863395158415),
 ('amazing', 0.09692895958028834),
 ('experience', 0.09688492519484725)]

Interestingly, the word "never" and bad" which may be negative words probably affect this


Let's look at other examples

In [40]:
incorrect_1.text[22539]

'wait reviewed crepes parisian penni first fell love crepes early teenage years crepe shop opened hometown state college yes town best known love football binge drinking restaurant sold crepes practically transcendent stuffed anything could imagine surprisingly last long heart broke closed could get crepe fix yearly family vacations disney world in sophomore year college studied abroad italy rediscovered crepe obsession found british style pub trip paris four days eating simple street version day breakfast returned state college crepe less sad excited moved pittsburgh found crepes paris ennis even think need answer questions these good crepes perfectly crispy outside soft inside topping list unfortunately limited many veggies one type cheese bacon breakfast crepe strawberries dessert crepe still cannot really go wrong combinations in came back state college tried new crepe shop town sucked made realize totally take crepes paris ennis granted food might simple trust say pittsburgh lucky

This specific review has a really long length. Maybe the incorrect assumption is tied to the token length of the sentence going beyond the context length of the model?

<br>

Anyway, we can see that there's some negative words here like "wrong", "cannot" and "heart broke"

In [42]:
get_top_attended_words(incorrect_1.text[22539], 'nlptown/bert-base-multilingual-uncased-sentiment', top_n=10)

[('many', 0.11025072930876728),
 ('less', 0.10924100640737915),
 ('sold', 0.1091226130129119),
 ('answer', 0.10675539588333818),
 ('cannot', 0.09768045302673808),
 ('questions', 0.09531632989643071),
 ('style', 0.09409198279839244),
 ('limited', 0.09360749617315424),
 ('need', 0.09037736056553787),
 ('last', 0.089730187899381)]

We do see that a lot of words like "less", "cannot" and "limited" might connotate to negative sentiment.

The more so neutral words like "sold" and "answer" may also slightly connotate towards that.

Let's look at one more last example before analysing the role of context length

In [43]:
incorrect_1.text[44001]

'atmosphere really nice server took forever know many tables service slow food bland overall tasteless many better alternatives phoenix area waste time'

Here we directly start with some really positive words but the review becomes more negative later on.

Let's see what the model decided to focus on!

In [44]:
get_top_attended_words(incorrect_1.text[44001], 'nlptown/bert-base-multilingual-uncased-sentiment', top_n=10)

[('atmosphere', 0.1389096469846583),
 ('know', 0.12247882523831836),
 ('phoenix', 0.1095007317115441),
 ('tables', 0.09957821914220037),
 ('server', 0.09844110710409718),
 ('service', 0.09725036371750918),
 ('##less', 0.0948395630559057),
 ('bland', 0.09448296326911168),
 ('really', 0.09295481919571039),
 ('many', 0.0917020695467984)]

As we expected, it did focus on atmosphere the most. But it also focuses a lot on pheonix, which is a city. Will it change the review if we remove the top 5 attention score words?

In [47]:
from transformers import pipeline

model_pipeline_1 = pipeline(model="distilbert-base-uncased-finetuned-sst-2-english", task="sentiment-analysis")

print(model_pipeline_1('really nice took forever many service slow food bland overall tasteless many better alternatives area waste time'))

[{'label': 'NEGATIVE', 'score': 0.9958256483078003}]


It does become negative! Let's look at the attention scores now.

In [48]:
get_top_attended_words('really nice took forever many service slow food bland overall tasteless many better alternatives area waste time', 'distilbert-base-uncased-finetuned-sst-2-english', top_n=10)

[('nice', 0.1271290696051829),
 ('service', 0.08754791983805009),
 ('bland', 0.08537290109112898),
 ('alternatives', 0.08195795190598412),
 ('took', 0.0702056008118682),
 ('many', 0.06820320562748192),
 ('really', 0.06695909363165373),
 ('area', 0.059745954109471665),
 ('waste', 0.058999269110081944),
 ('food', 0.05485497491754285)]

We can see that a lot of negative words are in the top 10 attended words.

In [41]:
from transformers import AutoTokenizer

def get_token_length(sentence, model_name):
    """
    Returns the length of the tokenized sentence using a pre-trained transformer model
    from Hugging Face.

    Args:
        sentence (str): The input sentence.
        model_name (str): The name of the pre-trained Hugging Face model to use.

    Returns:
        int: The length of the tokenized sentence.
    """
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Tokenize the input sentence
    tokens = tokenizer.tokenize(sentence)

    # Return the length of the tokenized sentence
    return len(tokens)

Let's sort the reviews by number of words first since more words = more tokens

In [51]:
incorrect_1_sorted = incorrect_1.assign(num_words=incorrect_1['text'].str.split().str.len()).sort_values('num_words', ascending=False)


In [52]:
incorrect_1_sorted

,label,text,ground_truth_sentiment,sentiment_analysis_1,num_words
4777,4,like people visit las vegas found high meth st...,positive,negative,504
46276,0,beginning a days las vegas soon arrived starte...,negative,positive,495
35240,3,sonnets done completed a days friday pretty go...,positive,negative,494
49607,0,watching top chef already michael volt agios s...,negative,positive,493
39447,3,great hotel price get cheap deals good rates o...,positive,negative,492
...,...,...,...,...,...
34379,0,good,negative,positive,1
34642,1,fast,negative,positive,1
36321,0,serve,negative,positive,1
15566,1,comment,negative,positive,1


Interestingly for this case, all the reviews are under the context length.

Let's move on to model 2



Let's first see the accuracy.

In [55]:
accuracy_model_2 = (combined['sentiment_analysis_2'] == combined['ground_truth_sentiment']).mean() * 100
accuracy_model_2


83.04464357148572

This model scores higher than the original model.

In [53]:
incorrect_2 = combined[combined['sentiment_analysis_2'] != combined['ground_truth_sentiment']]
incorrect_2.drop(columns=['sentiment_analysis_1', 'sentiment_analysis_3'], axis=1, inplace=True)
incorrect_2 = incorrect_2.reindex(columns=['label','text', 'ground_truth_sentiment', 'sentiment_analysis_2'])

/tmp/ipykernel_50355/1819985054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incorrect_2.drop(columns=['sentiment_analysis_1', 'sentiment_analysis_3'], axis=1, inplace=True)


In [54]:
incorrect_2

,label,text,ground_truth_sentiment,sentiment_analysis_2
2,0,new mexico thought id go amazing green red chi...,negative,positive
11,1,recent trip vegas girlfriend decided stop quic...,negative,positive
12,0,first good where began cannot begin known know...,negative,positive
15,1,customer service best neither bagels no bagels...,negative,positive
16,1,tried havana cafe several times love cuban foo...,negative,positive
...,...,...,...,...
49976,1,reservation made recommendation concierge new ...,negative,positive
49981,1,course overly impressed quality actual course ...,negative,positive
49984,1,salon star rating based salon my friend gift c...,negative,positive
49988,1,something old spaghetti factory love really li...,negative,positive


Right off the bat we can see that a lot of these reviews are actually negative in ground truth.

In [56]:
incorrect_2.ground_truth_sentiment.value_counts()

ground_truth_sentiment
negative    5241
positive    3236
Name: count, dtype: int64

We do have more negative reviews than positive ones, but not by a very large margin. 

Let's look at the negative reviews first.

In [58]:
incorrect_2.sample(5)

,label,text,ground_truth_sentiment,sentiment_analysis_2
35145,1,better yams mac a cheese chicken waffles good ...,negative,positive
27129,1,went sunday usually go chain restaurants simpl...,negative,positive
48751,4,id always gone tempe location one day drove on...,positive,negative
45272,1,located inconspicuous office park summer in vi...,negative,positive
44712,1,worse better remember coming hotel new excitin...,negative,positive


In [59]:
incorrect_2.text[35145]

'better yams mac a cheese chicken waffles good think change taste burn sugars'

Interestingly, that does not look like a negative review at first sight at all. Perhaps stopword removal is a reason?

Let's look at the attention scores

In [62]:
get_top_attended_words(incorrect_2.text[35145], model_2, top_n=10)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('Ġburn', 0.12429455186672082),
 ('Ġtaste', 0.11256080750536215),
 ('Ġchange', 0.08949126762571621),
 ('better', 0.08711572451654356),
 ('Ġgood', 0.08327986972418937),
 ('Ġsugars', 0.08302358181691519),
 ('Ġchicken', 0.07897849968250942),
 ('Ġthink', 0.07223392039142912),
 ('Ġw', 0.06453572549630987),
 ('Ġmac', 0.06351876395081035)]

Interestingly we see that a lot of words are actually positive, like good, sugar. Perhaps the names of food are also positive? 

Let's look at other examples

In [63]:
incorrect_2.text[27129]

'went sunday usually go chain restaurants simply many delicious locally owned restaurants area however gift card decided go dinner new got around a wait table however left lot people lobby waiting seat new ordered glass house red well priced good house wine in ordered soup salad pasta ago i soup husband ordered lasagna primavera lighter side part menu spinach veggies in breadsticks salad came like expected breadsticks always relish hard turn away eating soup good husbands lasagna even better however joking wonder lasagna lighter side small guess expected given making lasagna light easy task cannot even begin explain small was and normally normally split meals restaurants typically used smaller portions begin probably size half deck cards width length height like a small pieces grilled chicken ate complaint tasty though normally skip dessert waitress came husband ordered pumpkin cheesecake sure still hungry craving something sweet good huge slice cheesecake tasty slice gone like a minut

This yet again is an interesting example. This specific review is longer than the previous one yet also does not focus more on the "bad" part of the review, but actually wants to tell a story. This means a large section of the review starts with a good experience and generally positive words like "many delicious" and "gift card"

<br>
Let's look at the top attended words

In [65]:
get_top_attended_words(incorrect_2.text[27129], model_2, top_n=20)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('went', 0.08571032425975739),
 ('Ġtypically', 0.078513232757435),
 ('Ġrestaurants', 0.07422453045531559),
 ('Ġwidth', 0.07202442480284946),
 ('Ġwaitress', 0.0705371983363471),
 ('Ġcannot', 0.06554504361397465),
 ('Ġspinach', 0.06439080891383735),
 ('Ġlength', 0.06430681773518801),
 ('Ġhusband', 0.06378154147668941),
 ('Ġmany', 0.0628289932149831),
 ('Ġdelicious', 0.06271431964261057),
 ('Ġexpected', 0.0626416168878576),
 ('Ġtasty', 0.061730227594658044),
 ('Ġhusband', 0.06043951710125839),
 ('Ġtasty', 0.059901138705779684),
 ('Ġwould', 0.05981449450669629),
 ('Ġsmall', 0.05837876797748246),
 ('Ġgo', 0.057640770086682856),
 ('Ġlight', 0.057113586400895935),
 ('Ġalways', 0.05584949157325051)]

This is very interesting, we see that the top words do have generally positive words but a lot of them are neutral words as well.

Perhaps this was a neutral review that converted into a negative one when we decided to convert all neutral reviews to negatives.

Let's look at some positive reviews now.

In [78]:
incorrect_2.sample(2)

,label,text,ground_truth_sentiment,sentiment_analysis_2
4425,1,well crew lucky snuff visit pools spa area spr...,negative,positive
47795,3,long day sight seeing boyfriend needed caffein...,positive,negative


In [79]:
incorrect_2.text[47795]

'long day sight seeing boyfriend needed caffeine gas failed us exactly location starbucks ended good thing spotted green logo hauled three lanes ended needed cash needed latte needed dinner spotted totes laughed name asian diner saw red sticker approval new saw yelp sticker figured fate tons pan asian fusion options sheet spicy spicy like this place kick love got veggie penang brown rice call hippie tons veggies spicy curry cooked perfectly snap peas eggplant large chunks bell peppers asparagus mixed tofu greatly appreciate fact equal ratio veggies usually asian spots give five million chucks onions leave much else the boyfriend got crispy beef oil tough crispy sweetness good combo spicy curry in bathrooms clean of discount students valid current id food good deal get ever want expand california come santa cruz first line'

This review also has a similar story. It tells a story and starts off negative with the couple finding a place, but then diverges.

Perhaps a lot of focus is given to words like "long" and "failed" which push the context to negative.

In [81]:
get_top_attended_words(incorrect_2.text[47795], model_2, top_n=10)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[('Ġgas', 0.07432944994676897),
 ('Ġspicy', 0.07354530013830957),
 ('Ġspicy', 0.07160242729595565),
 ('Ġlove', 0.07013955698696211),
 ('long', 0.06864266490295595),
 ('Ġfood', 0.06822287412818974),
 ('Ġfailed', 0.06698665213891924),
 ('Ġcome', 0.066545486130772),
 ('Ġcurrent', 0.06334794807115109),
 ('Ġappreciate', 0.060585563619699516)]

 and Interestingly, it is easy to see why spicy may be a word with a negative connotation. Again, we do see "failed"  and "long" in the mix, and it makes sense why the model made this decision.


 Let's look at the last model, model 3 now

In [82]:
incorrect_3 = combined[combined['sentiment_analysis_3'] != combined['ground_truth_sentiment']]
incorrect_3.drop(columns=['sentiment_analysis_1', 'sentiment_analysis_2'], axis=1, inplace=True)
incorrect_3 = incorrect_3.reindex(columns=['label','text', 'ground_truth_sentiment', 'sentiment_analysis_3'])

/tmp/ipykernel_50355/1808121878.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incorrect_3.drop(columns=['sentiment_analysis_1', 'sentiment_analysis_2'], axis=1, inplace=True)


Let's look at the accuracy.

In [87]:
accuracy_model_3 = (combined['sentiment_analysis_3'] == combined['ground_truth_sentiment']).mean() * 100
accuracy_model_3

88.10504840387232

Again, this model does even better than the last one!

In [83]:
incorrect_3

,label,text,ground_truth_sentiment,sentiment_analysis_3
11,1,recent trip vegas girlfriend decided stop quic...,negative,positive
15,1,customer service best neither bagels no bagels...,negative,positive
37,1,usually come group people service tends good i...,negative,positive
42,3,writing first review customer service manageme...,positive,negative
43,1,visiting vegas friend looked see around yelp p...,negative,positive
...,...,...,...,...
49928,4,of sweet jesus manger really first thing thoug...,positive,negative
49936,1,good experience tonight make reservations got ...,negative,positive
49950,3,noisy crowded unorganized cares vegas food goo...,positive,negative
49984,1,salon star rating based salon my friend gift c...,negative,positive


In [86]:
incorrect_3.ground_truth_sentiment.value_counts()

ground_truth_sentiment
negative    4254
positive    1693
Name: count, dtype: int64

This model predicted negatives reviews wrongly by a landslide, compared to positives. This is possibly indicative of a stronger pattern.
Let's see what goes wrong!

<br>

We'll again take a random sample of the data.

In [88]:
incorrect_3.sample(5)

,label,text,ground_truth_sentiment,sentiment_analysis_3
18519,1,restaurant located shopping center hosting num...,negative,positive
13815,0,bell agio buffet good whelming amount food col...,negative,positive
39981,0,service excellent food salty good asian cafe e...,negative,positive
43368,1,went brazilian bull steakhouse couple years ag...,negative,positive
39265,1,shortage bbl sauce today thanks no barbecue ch...,negative,positive


In [89]:
incorrect_3.text[18519]

'restaurant located shopping center hosting number restaurants including wildly popular firefly shares ownership dragonfly unfortunately asian bistro concept fallen deaf wallets people bar restaurant empty of pm monday evening contrast standing room firefly wife ordered thai beef salad pickled cucumber warm spicy sauce excellent ordered banana leaf salmon steamed miso shiitake mushrooms atlantic salmon good obviously steamed banana leaf cold touch good side dish baby book chop stir fried oyster mushrooms shared'

From first glance, this does not look to be a bad review at all. But the score says otherwise.

In [90]:
incorrect_3.text[13815]

'bell agio buffet good whelming amount food cold greasy tasteless bell agio cafe would highly recommend food buffet personally would rather one well prepared meal eat mediocre food'

This review uses verbiage like "highly recommend". This possibly could be the reason why it did not detect the overall negative tone with "cold greasy tasteless" and "mediocre"

<br>

Let's look at the top attended words.

In [91]:
get_top_attended_words(incorrect_3.text[13815], model_3, top_n=10)

[('Ġmediocre', 0.061311332866167634),
 ('Ġgood', 0.05627847767293439),
 ('Ġwould', 0.05095865168959703),
 ('Ġtast', 0.049933617669944856),
 ('Ġone', 0.048325347241612054),
 ('Ġcafe', 0.04787384716833188),
 ('Ġpersonally', 0.04644331020283121),
 ('eless', 0.04604194138393325),
 ('Ġcold', 0.04581967607931845),
 ('Ġfood', 0.04438480896786288)]

We can see that mediocre is the highest attended word. But also that the variance in attention is very spread out, there's not much difference between the attention scores of the first and the 10th word. This means that it may just come down to the number of positive and negative words.

Let's look at how certain the model is while predicting the score.

In [97]:
model_pipeline_3 = pipeline(model=model_3, task="sentiment-analysis")
model_pipeline_3(incorrect_3.text[13815])

[{'label': 'POSITIVE', 'score': 0.9944918155670166}]

Surprisingly, it is very certain that this is a positive review.

Perhaps for this specific model or example, the attention score does not necessarily correlate highly? Or maybe it has a very black-and-white view.


Let's try testing it with a balanced sentence to see what happens. Facts are usually neutral sentences.

In [98]:
model_pipeline_3("The weather today is cloudy.")

[{'label': 'NEGATIVE', 'score': 0.9972461462020874}]

Interestingly, it gives an overwhelmingly negative score.

In [99]:
model_pipeline_3("The book is hardcover and has 300 pages.")

[{'label': 'NEGATIVE', 'score': 0.9927911758422852}]

Again, a very negative response to a very neutral sentence.

In [100]:
model_pipeline_3("The car is blue in color.")

[{'label': 'NEGATIVE', 'score': 0.9941356182098389}]

In [101]:
model_pipeline_3("The car is golden in color.")

[{'label': 'POSITIVE', 'score': 0.9979481101036072}]

In [103]:
model_pipeline_3("The car is red in color.")

[{'label': 'POSITIVE', 'score': 0.8135625123977661}]

We can see that simply changing the color changes the output! Maybe because golden is more attributed with "gold" and positivity. 
<br><br>
Even red shows a positive sign. Red is a color which is in positive and negative connotations, and we see that the model does give a slightly less polar score (0.81)



We realise from this that the model will jump to extremes and adheres too strongly to certain specific words. Maybe, it's slightly overfit while fine tuning. However, it's still the best model we saw with ~88% accuracy!